In [1]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

from ailab_tools.utils import split_and_stack

In [4]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

190606


味見　中身の確認

In [2]:
#df = pd.read_csv("/home/yusuke_akada/notebook/Drawbridge/device/part-00182_after.txt", header=None , names=['column'])
df = pd.read_csv("/home/yusuke_akada/notebook/adhoc/device/part-00000_after.txt", header=None , names=['column'])

df.head()

,column
0,1407000232427482|android_phone|drawbridge_cons...
1,1405000199612114|android_phone|drawbridge_cons...
2,1462000004321097|windows_computer|drawbridge_c...
3,2182000087470146|windows_computer|drawbridge_c...
4,2206000172608873|windows_computer|drawbridge_c...


In [3]:
splitted=df['column'].str.split('[|:]', expand=True)
splitted.columns = ['A', 'B','C','D']
del splitted['C']
splitted.columns = ['smn_uid', 'device','drawbridge_consumer_id']
stacked = split_and_stack(splitted, "smn_uid", sep="|")
stacked = stacked[['drawbridge_consumer_id','device','smn_uid']]
stacked.head()

,drawbridge_consumer_id,device,smn_uid
0,001d5e82286be61794e2a46662107018,android_phone,1407000232427482
1,001d5e82286be61794e2a46662107018,android_phone,1405000199612114
2,002ccb97b4583053fd158374241cab38,windows_computer,1462000004321097
3,002ccb97b4583053fd158374241cab38,windows_computer,2182000087470146
4,002ccb97b4583053fd158374241cab38,windows_computer,2206000172608873


分割されたテキストファイルを縦持ち＆統合しImpalaにUp

In [7]:
%%slack_notify @yusuke_akada
uid_table = "segment.drawbridge_device"   # tapad_idとsmn_uidの最終的な対応table
temp_table = f"dm_tmp.ya_temp_drawbridge_device_{today_yymmdd}"    # 分割処理用のtemporary table 実行後dropされる


with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:



    # テキストファイル単体ごとに読込、DataFrame化
    for i in range(0,200):
            if i < 10:
                df = pd.read_csv(f"/home/yusuke_akada/notebook/adhoc/device/part-0000{i}_after.txt", header=None, names=['column'])
            elif 10 <= i < 100:
                df = pd.read_csv(f"/home/yusuke_akada/notebook/adhoc/device/part-000{i}_after.txt", header=None, names=['column'])
            else:
                df = pd.read_csv(f"/home/yusuke_akada/notebook/adhoc/device/part-00{i}_after.txt", header=None, names=['column'])
                
            splitted=df['column'].str.split('[|:]', expand=True)
            splitted.columns = ['A', 'B','C','D']
            del splitted['C']
            splitted.columns = ['smn_uid', 'device','drawbridge_consumer_id']
            stacked = split_and_stack(splitted, "smn_uid", sep="|")
            stacked = stacked[['drawbridge_consumer_id','device','smn_uid']]
        
        # ImpalaにUP
            ir.upload_dataframe(temp_table, stacked, ['string','string','bigint'])
        
        # 親テーブルにInsert
            query = (
                f"""
                insert overwrite table {uid_table}
                    (
                    drawbridge_consumer_id
                    ,device
                    ,smn_uid 
                    )

                partition
                    (
                    block = {i}
                    )

                select
                 drawbridge_consumer_id
                 ,device
                 ,smn_uid
                from {temp_table}
                """
            )
            ir.execute_query_no_ret(query) 

            # tempテーブルdrop
            ir.drop_tables([temp_table])

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:Create empty 

以下削除

    #まず空tableを作る
/*    query = (
        f"""
        create table {uid_table}
            (
            drawbridge_consumer_id string
            ,device string
            ,smn_uid bigint
            )
        
        partitioned by 
            (
            block int
            )
        
        stored as parquet
        """
    )
    
    ir.execute_query_no_ret(query) 
*/